# <div align="center" style="color: #ff5733;">Demograpic Score Retrain DataSet</div>

**Date**    -   2024-09-25 <br>
**Created By**  -   Dwaipayan Chakroborti <br>

# Declare Libraries

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
pd.set_option('display.max_columns', None)

# Query

In [3]:
sq = """    
with 
master as (
select 
lmt.digitalLoanAccountId,
lmt.loanAccountNumber,
lmt.customerId,
lmt.new_loan_type,
lmt.startApplyDateTime,
lmt.disbursementDateTime,
lmt.termsAndConditionsSubmitDateTime,
lmt.tsa_onboarding_time,
cast(lmt.credolabScore as float64) as credolabScore,
----------------
ROUND(TIMESTAMP_DIFF(lmt.termsAndConditionsSubmitDateTime, lmt.tsa_onboarding_time, SECOND) / 3600.0, 2) AS tsa_time,
--------------------
cast(lmt.monthlyIncome as float64) as monthlyIncome,
--------------------
SPLIT(lmt.osVersion, '.')[OFFSET(0)] AS mainVersion,
--------------------
lmt.mobileNo,
---------------------
 case when lower(coalesce(lmt.osVersion, lmt.osversion_v2)) like '%ios%' and coalesce(cast(SUBSTRING(coalesce(lmt.osVersion, lmt.osversion_v2), 4, 2)  as numeric),0) in (0,12,13,14,15) then 'G1'
       when lower(coalesce(lmt.osVersion, lmt.osversion_v2)) like '%andro%' and coalesce(cast(SUBSTRING(coalesce(lmt.osVersion, lmt.osversion_v2), 8, 2)  as numeric),0) in (0,10, 11, 6, 7, 8, 9) then 'G1'
       when lower(coalesce(lmt.osVersion, lmt.osversion_v2)) like '%andro%' and coalesce(cast(SUBSTRING(coalesce(lmt.osVersion, lmt.osversion_v2), 8, 2)  as numeric),0) in (12) then 'G2'
       when lower(coalesce(lmt.osVersion, lmt.osversion_v2)) like '%andro%' and coalesce(cast(SUBSTRING(coalesce(lmt.osVersion, lmt.osversion_v2), 8, 2)  as numeric),0) in (13, 14) then 'G3'
       when lower(coalesce(lmt.osVersion, lmt.osversion_v2)) like '%ios%' and coalesce(cast(SUBSTRING(coalesce(lmt.osVersion, lmt.osversion_v2), 4, 2)  as numeric), 0) in (16,17) then 'G4'
       else 'G5' end deviceversion,
--------------------------
 CAST(SUBSTR(CAST(right(lmt.mobileNo, 10) AS STRING), 3, 3) AS INT64) mobileinitials,
---------------------------
lmt.subIndustryDescription,
--------------------------
lmt.loanRequestTenure,
-------------------------
case when ldd.obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end FSPD30
from risk_credit_mis.loan_master_table lmt
inner join prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data ldd on ldd.loanAccountNumber = lmt.loanAccountNumber
where lmt.flagDisbursement = 1
and lmt.new_loan_type in ('SIL-Instore', 'Quick')
)
--------------------------------------------------
select distinct  
 w.*
from (
    select
    digitalLoanAccountId,
    loanAccountNumber,
    customerId,
    new_loan_type,
    startApplyDateTime,
    disbursementDateTime,
    termsAndConditionsSubmitDateTime,
    tsa_onboarding_time,
    FSPD30,
    credolabScore,
    case when credolabScore is null then 'd. NULL'
         when credolabScore < 416.5 then 'a. <416.5'
         when credolabScore >= 416.5 and credolabScore < 472.5 then 'b. 416.5 - 472.5'
         when credolabScore >= 472.5 then 'c. >472.5'
         end credolabScore_bin_SIL,
    case when credolabScore is null then 'e. NULL'
         when credolabScore < 450.5 then 'a. <450.5'
         when credolabScore >= 450.5 and credolabScore < 460.5 then 'b. 450.5 - 460.5'
         when credolabScore >= 460.5 and credolabScore < 528.5 then 'c. 460.5 - 528.5'
         when credolabScore >= 528.5 then 'd. >=528.5' end credolabScore_bin_Cashloan,
    monthlyIncome,
    CASE
            WHEN coalesce(CAST(monthlyIncome AS FLOAT64), -1) = -1 then 'e. NULL'
            WHEN coalesce(CAST(monthlyIncome AS FLOAT64), -1) > -1 and coalesce(CAST(monthlyIncome AS FLOAT64), -1) < 15830.0 THEN 'a. 0 - 15830'
            WHEN coalesce(CAST(monthlyIncome AS FLOAT64), -1) >= 15830.0 and coalesce(CAST(monthlyIncome AS FLOAT64), -1) < 24750.0 THEN 'b. 15830 - 24750'
            WHEN coalesce(CAST(monthlyIncome AS FLOAT64), -1) >= 24750.0 and coalesce(CAST(monthlyIncome AS FLOAT64), -1) < 25650.0 THEN 'c. 24750 - 25650'
            WHEN coalesce(CAST(monthlyIncome AS FLOAT64), -1) >= 25650.0 THEN 'd. 25650 - inf'
            END AS monthlyIncomeBin_sil,
    CASE
            WHEN coalesce(CAST(monthlyIncome AS FLOAT64), -1) = -1 then 'e. NULL'
            WHEN coalesce(CAST(monthlyIncome AS FLOAT64), -1) > -1 and coalesce(CAST(monthlyIncome AS FLOAT64), -1) < 15830.0 THEN 'a. 0 - 15830'
            WHEN coalesce(CAST(monthlyIncome AS FLOAT64), -1) >= 15830.0 and coalesce(CAST(monthlyIncome AS FLOAT64), -1) < 24750.0 THEN 'b. 15830 - 24750'
            WHEN coalesce(CAST(monthlyIncome AS FLOAT64), -1) >= 24750.0 and coalesce(CAST(monthlyIncome AS FLOAT64), -1) < 25650.0 THEN 'c. 24750 - 25650'
            WHEN coalesce(CAST(monthlyIncome AS FLOAT64), -1) >= 25650.0 THEN 'd. 25650 - inf'
            END AS monthlyIncomeBin_cashloan,
     mobileinitials,
     case when mobileinitials in (961,962,963,964,968,922,923,924,925,931,932,933,934,940,941,942,943,944,973,974,991,993,895,896,897,898,992,994,960,965,966,967,969,970,981,985,987)
          then 'a. DITO, DITO or Globe or TM, DITO or Smart or Talk, Smart, Smart or TNT, Smart or Talk N Text, Sun Cellular, Unknown' 
          when mobileinitials in (817,904,905,906,915,916,917,926,927,935,936,945,953,954,955,956,975,976,977,978,979,995,996,997) then 'b. Smart or Talk N Text'
          when mobileinitials in (813,907,908,909,910,911,912,913,914,918,919,920,921,928,929,930,938,939,946,947,948,949,950,951,998,999) then 'c. Globe or TM'
          else 'd. Unknown Values' end telco_name_bin,
     deviceversion,
     case when deviceversion = 'G1' then 'android10, android11, android6, android7, android8, android9, ios12, ios13, ios14, ios15, na'
          when deviceversion = 'G2' then 'android12'
          when deviceversion = 'G3' then 'android13, android14'
          when deviceversion = 'G4' then 'ios16, ios17'
          when deviceversion = 'G5' then 'Unknown Value' end devicevrs_bin ,
      case
        when tsa_time is null then 'd. NULL'
        when coalesce(tsa_time, -1) >= 0 and coalesce(tsa_time, -1) < 0.195 then 'a. 0 - 0.195'
        when coalesce(tsa_time, -1) >= 0.195 and coalesce(tsa_time, -1) < 0.285 then 'b. 0.195 - 0.285'
        when coalesce(tsa_time, -1) >= 0.285 then 'c. >= 0.285' end tsa_time_bin_cashloan,
      case
        when tsa_time is null then 'e. NULL'
        when coalesce(tsa_time, -1) >= 0 and coalesce(tsa_time, -1) < 0.085 then 'a. 0 - 0.085'
        when coalesce(tsa_time, -1) >= 0.085 and coalesce(tsa_time, -1) < 0.165 then 'b. 0.085 - 0.165'
        when coalesce(tsa_time, -1) >= 0.165 and coalesce(tsa_time, -1) < 0.985 then 'c. 0.165 - 0.985' 
        when coalesce(tsa_time, -1) >= 0.985 then 'd. >0.985'
        end tsa_time_bin_sil,
      loanRequestTenure,
      CASE
          WHEN loanRequestTenure IS NULL THEN 'e. nan'
          WHEN loanRequestTenure < 4.5 THEN 'a. [-inf, 4.5)'
          WHEN loanRequestTenure < 7.5 THEN 'b. [4.5, 7.5)'
          WHEN loanRequestTenure < 10.5 THEN 'c. [7.5, 10.5)'
          ELSE 'd. [10.5, inf)'
        END AS loanRequestTenureBin_sil,
      subIndustryDescription,
 case when lower(subIndustryDescription) like 'call%' then 'a. CallCenter/BPO'
        when lower(subIndustryDescription) like 'other' then 'c.. other'
        when lower(subIndustryDescription) like 'gover%' then 'd. Government Departments / Institution'
        when subIndustryDescription like 'e-Commerce' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Construction' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Hospital' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Textile / Garment' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Media / News' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Passenger 4 wheeler (car)' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Food Processing' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Aviation' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'FMCG' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Technology' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Taxi Service; Transport Aggregator (Grab, Uber etc)' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Restaurant (Home Delivery / Cloud Kitchen)' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'White Goods' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Hotel / Resort / Lodge' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Agriculture /  Horticulture' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Banking' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Auto Ancillary' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Life / Medical Insurance' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Real Estate - Commercial / Residential' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Computer / Computer Peripherals' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Personal Healthcare/Hygiene' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Casino / Gambling' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Start Up Company (FinTech / Others)' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Manufacturing' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Grooming (Salon / Parlour)' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Education / School / College / Online Coaching' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Retail / Mall / Hyper Market' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Digital Payment / Wallets' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Co-Working Space / Business Centre' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Oil and Gas / Ancillary (RIG Installation / Maintenance)' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Entertainment (Cinema Halls / Event Management)' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Telecom / Mobile Telephony' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Restaurant / Foodcourt / Cafe (brick and mortar) / Pubs' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Finance Company / Consumer Finance Company' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Retail (Super market / Grocery)' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Fitness (Gym)' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Cargo Transport' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Pharmaceutical' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Pawn Shops' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Medical Equipment' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Office Furnishing / Fitout' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Government Institution' then 'e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Trading' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Logistics / Warehousing / Cold Storage' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'IT / Tech Consultancy / Projects / AI ML Solution Providers' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Travel & Tourism' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Manpower / Manning Agencies (Construction Industry)' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Marketing / Advertisement (TV / Paper / Radio)' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Services' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Agriculture' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
when subIndustryDescription like 'Financial Services' then 'b. e-CommerceConstructionHospitalTextile / GarmentMedia / NewsPassenger 4 wheeler (car)Food ProcessingAviationFMCGTechnologyTaxi Service; Transport Aggregator (Grab, Uber etc)Restaurant (Home Delivery / Cloud Kitchen)White GoodsHotel / Resort / LodgeAgriculture /  HorticultureBankingAuto AncillaryLife / Medical InsuranceReal Estate - Commercial / ResidentialComputer / Computer PeripheralsPersonal Healthcare/HygieneCasino / GamblingStart Up Company (FinTech / Others)ManufacturingGrooming (Salon / Parlour)Manpower / Manning Agencies(Security Services; House Keeping; Logistics / Ports)Building and Construction (e.g. cement, steel, paint, brick, tiles, interior decoration, bathroom fitouts, elevator manufactures, residential grade glass makers)Education / School / College / Online CoachingRetail / Mall / Hyper MarketDigital Payment / WalletsCo-Working Space / Business CentreOil and Gas / Ancillary (RIG Installation / Maintenance)Entertainment (Cinema Halls / Event Management)Telecom / Mobile TelephonyRestaurant / Foodcourt / Cafe (brick and mortar) / PubsFinance Company / Consumer Finance CompanyRetail (Super market / Grocery)Fitness (Gym)Cargo TransportPharmaceuticalPawn ShopsMedical EquipmentOffice Furnishing / FitoutGovernment InstitutionTradingLogistics / Warehousing / Cold StorageIT / Tech Consultancy / Projects / AI ML Solution ProvidersTravel & TourismManpower / Manning Agencies (Construction Industry)Marketing / Advertisement (TV / Paper / Radio)ServicesAgricultureFinancial Services'
else 'e. Unknown Value' end subIndustryDescriptionbin_sil,
 
    from master
) w

"""

# Execute the query and save to DF

In [4]:
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')
print(f"The shape of the dataframe is:\t {df.shape}")

Job ID c79d86d0-77fc-440f-8072-6b036d6ad08f successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (157185, 25)


In [5]:
df.head()

,digitalLoanAccountId,loanAccountNumber,customerId,new_loan_type,startApplyDateTime,disbursementDateTime,termsAndConditionsSubmitDateTime,tsa_onboarding_time,FSPD30,credolabScore,credolabScore_bin_SIL,credolabScore_bin_Cashloan,monthlyIncome,monthlyIncomeBin_sil,monthlyIncomeBin_cashloan,mobileinitials,telco_name_bin,deviceversion,devicevrs_bin,tsa_time_bin_cashloan,tsa_time_bin_sil,loanRequestTenure,loanRequestTenureBin_sil,subIndustryDescription,subIndustryDescriptionbin_sil
0,19b4d3db-7327-4892-b867-9e292ac6b495,60826202650015,2620265,SIL-Instore,2024-06-29 18:58:57,2024-06-29 19:06:49,2024-06-29 19:03:49,2024-06-29 18:56:24,0,NaN,d. NULL,e. NULL,70000.0,d. 25650 - inf,d. 25650 - inf,607,d. Unknown Values,G4,"ios16, ios17",a. 0 - 0.195,b. 0.085 - 0.165,3,"a. [-inf, 4.5)",Others,e. Unknown Value
1,90a6964a-e6ab-4235-a69c-76f74c03d1b6,60826665650018,2666565,SIL-Instore,2024-07-17 10:49:34,2024-07-17 11:01:39,2024-07-17 10:58:01,2024-07-17 10:44:01,0,437.0,b. 416.5 - 472.5,a. <450.5,24000.0,b. 15830 - 24750,b. 15830 - 24750,636,d. Unknown Values,G1,"android10, android11, android6, android7, andr...",b. 0.195 - 0.285,c. 0.165 - 0.985,3,"a. [-inf, 4.5)",Others,e. Unknown Value
2,4dd566e7-1b01-4082-a261-32a6573a00ba,60827593660016,2759366,SIL-Instore,2024-08-16 12:50:36,2024-08-16 13:12:57,2024-08-16 12:52:45,2024-08-16 12:47:05,0,373.0,a. <416.5,a. <450.5,50000.0,d. 25650 - inf,d. 25650 - inf,669,d. Unknown Values,G2,android12,a. 0 - 0.195,b. 0.085 - 0.165,3,"a. [-inf, 4.5)",None,e. Unknown Value
3,62064d6f-d2b2-46b8-8249-13df6bf2de83,60822645290012,2264529,SIL-Instore,2023-10-08 10:58:33,2023-10-08 11:06:48,2023-10-08 11:02:52,2023-10-08 10:51:20,1,461.0,b. 416.5 - 472.5,c. 460.5 - 528.5,50000.0,d. 25650 - inf,d. 25650 - inf,542,d. Unknown Values,G3,"android13, android14",a. 0 - 0.195,c. 0.165 - 0.985,3,"a. [-inf, 4.5)",None,e. Unknown Value
4,02f70b18-6bc9-4114-a81b-eedd0cb62d5b,60826833620017,2683362,SIL-Instore,2024-07-23 10:45:16,2024-07-23 10:57:24,2024-07-23 10:49:30,2024-07-23 10:43:03,0,436.0,b. 416.5 - 472.5,a. <450.5,15000.0,a. 0 - 15830,a. 0 - 15830,76,d. Unknown Values,G3,"android13, android14",a. 0 - 0.195,b. 0.085 - 0.165,3,"a. [-inf, 4.5)",Others,e. Unknown Value


# Save the file to csv file

In [6]:
df.to_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\Demographic_Retrain\Data\Demographic_score_retrain_20240925_V1.csv", index = False)

In [8]:
df.to_pickle(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\Demographic_Retrain\Data\demographic_score_retain_sql_v1.pkl")